# Modal Analysis of a Console in 3D

A short comparison of different modeling techniques of a simple console. The beam has a rectangular prismatic cross-section and a linear elastic material model, governed by the following parameters:

In [1]:
# all units in kN and cm
L = 150.  # length of the console [cm]
F = 1.  # value of the vertical load at the free end [kN]
E = 21000.0  # Young's modulus [kN/cm3]
nu = 0.3  # Poisson's ratio [-]
density = 7750.0 * 1e-6 # mass density [g/cm3]
weight = density * 1e-3 * 9.81  # [kN/cm3]
w, h = 5., 15.  # width and height of the rectangular cross section
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = h * w**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia
nElem = 20  # number of subdivisons to use

## Continuous density distribution

In [2]:
dpa = density * A  # g/cm
wpa = weight * A  # kN/cm

In [3]:
from sigmaepsilon.solid import Structure, LineMesh, PointData
from neumann.linalg import linspace, Vector
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from sigmaepsilon.solid.fem.cells import B2 as Beam

import numpy as np
from numpy import pi as PI


# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)

# mesh
p0 = np.array([0., 0., 0.])
p1 = np.array([L, 0., 0.])
coords = linspace(p0, p1, nElem+1)
coords = PointCloud(coords, frame=GlobalFrame).show()
topo = np.zeros((nElem, 2), dtype=int)
topo[:, 0] = np.arange(nElem)
topo[:, 1] = np.arange(nElem) + 1

# load at the rightmost node in Y and Z directions
loads = np.zeros((coords.shape[0], 6, 3))
global_load_vector = Vector([F,  0, 0.], frame=GlobalFrame).show()
loads[-1, :3, 0] = global_load_vector
global_load_vector = Vector([0., F, 0.], frame=GlobalFrame).show()
loads[-1, :3, 1] = global_load_vector
global_load_vector = Vector([0., 0, F], frame=GlobalFrame).show()
loads[-1, :3, 2] = global_load_vector

# support at the leftmost node (all degrees)
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
fixity[0, :] = True

# mass and density
nodal_masses = np.zeros((coords.shape[0],))
densities = np.full((topo.shape[0],), dpa)

# pointdata
pd = PointData(coords=coords, loads=loads, fixity=fixity, mass=nodal_masses)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames, density=densities)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linsolve()

# postproc
dofsol = structure.nodal_dof_solution(store='dofsol')

In [4]:
dofsol[-1, :3, 0], dofsol[-1, :3, 1], dofsol[-1, :3, 2]

(array([ 9.52380962e-05,  0.00000000e+00, -0.00000000e+00]),
 array([ 0.        ,  0.03809526, -0.        ]),
 array([0.        , 0.        , 0.34285717]))

In [5]:
structure.consistent_mass_matrix()
freks, modes = structure.natural_circular_frequencies(normalize=True, as_dense=True, 
                                                      return_vectors=True)
from sigmaepsilon.solid.fem.dyn import estimate_smallest_natural_circular_frequency

M = structure.Solver.M
u = structure.Solver.u
f = structure.Solver.f
f = f.reshape(u.shape)
f_Rayleigh = estimate_smallest_natural_circular_frequency(M, u=u, f=f)
f_Rayleigh, min(f_Rayleigh)

(array([19.00763833,  1.12837795,  0.37612612]), 0.3761261160830689)

In [6]:
around = min(f_Rayleigh)
freks, modes = structure.natural_circular_frequencies(k=40, around=around, 
                                                      return_vectors=True, which='LM')
freks[np.where(np.abs(freks) < 1e-3)] = 0.
freks[:40]

array([  0.37074595,   1.11223747,   2.30356896,   6.36330294,
         6.91070448,  12.23318805,  17.24245204,  19.08990228,
        19.74781554,  28.69482786,  36.69955178,  38.86115579,
        48.80807753,  50.04593837,  51.83376943,  59.24342703,
        62.07050752,  74.78335756,  86.08445571,  86.74493763,
        88.06136234, 101.80842611, 115.95252313, 116.5834305 ,
       122.19100059, 130.44161272, 145.23822125, 146.72545594,
       150.13776874, 158.38887523, 160.31110692, 175.61843383,
       186.21146639, 191.06116324, 195.55622439, 206.30181625,
       219.75384894, 224.35000651, 233.90797533, 245.54823346])

In [7]:
modes.shape

(126, 40)

In [8]:
dofsol.shape

(21, 6, 3)

In [9]:
from sigmaepsilon.solid.fem.dyn import effective_modal_masses

nN, nD, nR = dofsol.shape

action_x = np.zeros((nN, nD))
action_x[:, 0] = 1.0
action_x = action_x.reshape(nN * nD)
action_y = np.zeros((nN, nD))
action_y[:, 1] = 1.0
action_y = action_y.reshape(nN * nD)
action_z = np.zeros((nN, nD))
action_z[:, 2] = 1.0
action_z = action_z.reshape(nN * nD)
actions = np.stack([action_x, action_y, action_z], axis=1)

m_eff = effective_modal_masses(M, actions, modes)
mtot = np.sum(m_eff[:, 0]), np.sum(m_eff[:, 1]), np.sum(m_eff[:, 2])
L * dpa, mtot

(87.1875, (84.65499360063335, 82.71567035317051, 84.04432398581964))

In [10]:
actions.shape

(126, 3)

In [11]:
m_eff[np.where(m_eff <= 1e-5)] = 0.
for i in range(40):
    print(m_eff[i], freks[i])

[ 0.          0.         53.29078559] 0.3707459526639891
[ 0.         53.29079498  0.        ] 1.1122374692527475
[ 0.          0.         16.33039797] 2.3035689592597293
[0.         0.         5.53858809] 6.363302940274247
[ 0.         16.33040088  0.        ] 6.910704480677076
[0.         0.         2.78493513] 12.233188054749498
[70.67150017  0.          0.        ] 17.242452035297333
[0.         5.53858914 0.        ] 19.089902275984368
[0.         0.         1.64981327] 19.747815543225315
[0.         0.         1.07829849] 28.6948278608562
[0.        2.7849357 0.       ] 36.6995517830835
[0.         0.         0.75217352] 38.86115579003838
[0. 0. 0.] 48.808077533956094
[0.         0.         0.54979067] 50.045938367626675
[7.85215518 0.         0.        ] 51.8337694300286
[0.         1.64981365 0.        ] 59.243427027334135
[0.         0.         0.41641496] 62.07050751801438
[0.         0.         0.32445726] 74.78335755760547
[0.         1.07829878 0.        ] 86.0844557147202

In [12]:
L * dpa, np.sum(structure.mesh.cd.lengths()) * dpa

(87.1875, 87.1875)

In [13]:
m_tot = L * dpa
["{:.2f}%".format(100 * m / m_tot) for m in mtot]

['97.10%', '94.87%', '96.39%']

## Dunkerley

In [14]:
N = 1
mi = m_tot / N
h = L / N
i3 = np.sum([i**3 for i in range(1, N+1)])
acc_y = i3 * (mi * h**3) / (3 * E * Iy)
acc_z = i3 * (mi * h**3) / (3 * E * Iz)
freq_lower_y, freq_lower_z = np.sqrt(1/acc_y), np.sqrt(1/acc_z)
freq_lower_y, freq_lower_z

(0.5487032611687344, 0.18290108705624478)

In [15]:
N = nElem
mi = m_tot / N
h = L / N
i3 = np.sum([i**3 for i in range(1, N+1)])
acc_y = i3 * (mi * h**3) / (3 * E * Iy)
acc_z = i3 * (mi * h**3) / (3 * E * Iz)
freq_lower_y, freq_lower_z = np.sqrt(1/acc_y), np.sqrt(1/acc_z)
freq_lower_y, freq_lower_z

(1.0451490688928273, 0.3483830229642758)

## Concentrate the mass to one point

### Analytic Solution

In [16]:
kx =  F / dofsol[-1, 0, 0]
ky =  F / dofsol[-1, 1, 1]
kz =  F / dofsol[-1, 2, 2]
np.sqrt(kx/m_tot), np.sqrt(ky/m_tot), np.sqrt(kz/m_tot)

(10.974065165760832, 0.5487030991197182, 0.1829010810559317)

In [17]:
ky_=3*E*Iy/L**3
kz_=3*E*Iz/L**3
(ky_, ky), (kz_, kz)

((26.25, 26.249984495131162), (2.9166666666666665, 2.9166664752964215))

### FEM - 20 Nodes

In [18]:
# mass and density
nN = coords.shape[0]
min_mass = m_tot / (100 * nN)
nodal_masses = np.full((coords.shape[0],), m_tot/nN)
densities = np.full((topo.shape[0],), min_mass)

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity, mass=nodal_masses)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames, density=densities)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linsolve()

# postproc
dofsol = structure.nodal_dof_solution(store='dofsol')

In [19]:
structure.consistent_mass_matrix()
freks, modes = structure.natural_circular_frequencies(normalize=True, as_dense=True, 
                                                      return_vectors=True)
M = structure.Solver.M
u = structure.Solver.u
f = structure.Solver.f
f = f.reshape(u.shape)

nN, nD, nR = dofsol.shape

action_x = np.zeros((nN, nD))
action_x[:, 0] = 1.0
action_x = action_x.reshape(nN * nD)
action_y = np.zeros((nN, nD))
action_y[:, 1] = 1.0
action_y = action_y.reshape(nN * nD)
action_z = np.zeros((nN, nD))
action_z[:, 2] = 1.0
action_z = action_z.reshape(nN * nD)
actions = np.stack([action_x, action_y, action_z], axis=1)

m_eff = effective_modal_masses(M, actions, modes)
mtot = np.sum(m_eff[:, 0]), np.sum(m_eff[:, 1]), np.sum(m_eff[:, 2])
L * dpa, mtot

m_eff[np.where(m_eff <= 1e-5)] = 0.
for i in range(10):
    print(m_eff[i], freks[i])

[ 0.          0.         56.00628429] 0.35056119653032214
[ 0.         56.00629411  0.        ] 1.0516832172774053
[ 0.          0.         17.19122614] 2.197986223009937
[0.         0.         5.90519796] 6.154844156489208
[ 0.         17.19122899  0.        ] 6.5939563468316615
[0.         0.         3.01440765] 12.057293499899133
[73.96910606  0.          0.        ] 16.64566732547229
[0.         5.90519878 0.        ] 18.464525995370547
[0.         0.         1.81991425] 19.917043385856623
[0.         0.         1.21508086] 29.716906672007102


In [20]:
f_Rayleigh = estimate_smallest_natural_circular_frequency(M, u=u, f=f)
f_Rayleigh, min(f_Rayleigh)

(array([18.15256447,  1.06431997,  0.35477344]), 0.3547734449222791)